# Programming for Data Science and Artificial Intelligence

## Supervised Learning - Classification - Bagging and Random Forests

### Readings: 
- [GERON] Ch7
- [VANDER] Ch5
- [HASTIE] Ch15
- https://scikit-learn.org/stable/modules/ensemble.html

### Lab09 - Assignment 

In [1]:
Name = "Muhammad Omer Farooq Bhatti"
Id = "122498"

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from scipy import stats

## Bagging

A single decision tree does not perform well as it tends to overfit.  A possible solution is the construct multiple trees to reduce variances.  To make sure each tree is not exactly learning the same thing since it will then be all same trees, we need to inject some differences to these trees (i.e., make them as diverse as possible but at the same time they also see some overlappinp samples).  One simple idea is that each of the tree is trained on a subset of **bootstrapping sample** and then perform some sort of aggregation of the decision.

The process has the following steps:

1. Sample $m$ times **with replacement** from the original training data
2. Repeat $B$ times to generate $B$ "boostrapped" training datasets $D_1, D_2, \cdots, D_B$
3. Train $B$ trees using the training datasets $D_1, D_2, \cdots, D_B$ 

Boostrapping the data plus performing some sort of aggregation (averaging or majority votes) is called **boostrap aggregation** or **bagging**.

*Example*:

Assume that we have a training set where $m=4$, and $n=2$:

$$D = {(x_1, y_1), (x_2, y_2), (x_3, y_3), (x_4, y_4)}$$

We generate, say, $B = 3$ datasets by boostrapping:

$$D_1 = {(x_1, y_1), (x_2, y_2), (x_3, y_3), (x_3, y_3)}$$
$$D_2 = {(x_1, y_1), (x_4, y_4), (x_4, y_4), (x_3, y_3)}$$
$$D_3 = {(x_1, y_1), (x_1, y_1), (x_2, y_2), (x_2, y_2)}$$

We can then train 3 trees.

Note: When sampling is performed **without** replacement, it is called **pasting**.  In other words, both bagging and pasting allow training instacnes to be sampled several times across multiple predictors, but only bagging allows training instances to be sampled several times for the same predictor.

Let's try to code from scratch.  To make our life easier, we shall use DecisionTree from the sklearn library (since we already code it from scratch in the previous class)

### ===Task===

#### Out of Bag Evaluation

Well, it seems like our bagging technique is quite good.  Anyhow, one interesting observation is that each tree only see a subset of the dataset. Any data that a particular tree did not see is called **out of bag** (oob).  Note that oob is not the same for all predictors.

One interesting thing is that since oob is something that each tree never see, thus oob is somewhat a validation set.  Thus what we can do is after we fit each tree. We can ask each tree to test their accuracy with their own oob, and then we can average the accuracy from all trees.  

Let's modify the above scratch code to:
- Calculate for oob evaluation for each bootstrapped dataset, and also the average score
- Change the code to "without replacement"
- Put everything into a class <code>Bagging</code>.  It should have at least two methods, <code>fit(X_train, y_train)</code>, and <code>predict(X_test)</code>
- Modify the code from above to randomize features.  Set the number of features to be used in each tree to be <code>sqrt(n)</code>, and then select a subset of features for each tree.  This can be easily done by setting our DecisionTreeClassifier <code>max_features</code> to 'sqrt'

## Random Forests

So far, it seems bagging works well.  However, the $B$ bootstrapped dataset are correlated, thus the power of variance reduction is diminished.  How do we further de-correlate these $B$ trees?

A **random forest** is constructed by bagging, but for each split in each tree, only a random subset of $q \leq n$ features are considered as splitting variables.

Rule of thumb: $q = \sqrt{n}$ for classification trees and $q = \frac{n}{3}$ for regression trees


1. https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
2. https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html
3. https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mode.html#scipy.stats.mode

In [4]:
class bagging:
    def __init__(self, b=5, bootstrap_ratio = 1.0, 
                 tree_parameters = {'max_depth':2, 'criterion':'gini', 'min_samples_split':5, 'max_features':'sqrt'}):
        #Defining the number of classifiers to be used
        self.B=b
        #Ratio of total training samples to be used to train a classifier
        self.bootstrap_ratio = bootstrap_ratio  #all of them

        #### CREATING DECISION TREE CLASSIFIERS FOR BAGGING
        #max_depth = depth of the tree
        #criterion = criteria for finding the best split of a node
        #min_samples_split = The minimum number of samples required to split an internal node
        self.classifiers = []
        for i in range(0,self.B):
            self.classifiers.append( DecisionTreeClassifier(**tree_parameters) )  #using keyword arguments

    def fit(self, X, y, with_replacement=True):
        #defining sample size in case bootstrap_ratio != 1
        self.sample_size = int(self.bootstrap_ratio * X.shape[0])
        print("Bagging with replacement: ", with_replacement)
        print("Using sample size: ", self.sample_size)
        #defining the matrices xsamples and ysamples which will hold the bootstrapped training data for all trees
        self.bootstrapped_xsamples = np.zeros((self.B, self.sample_size, X.shape[1]))
        self.bootstrapped_ysamples = np.zeros((self.B, self.sample_size))
        
        #sample data from X and y and fill bootstrap matrices, returns indexes of not used training samples
        notused_idx = self._bootstrap_samples(X, y, with_replacement)
        print("not used indexes: ", notused_idx)
              
        #Fitting dTrees to training samples and doing Out of Bag Evaluation
        accuracy=[]
        for i, classifier in enumerate(self.classifiers):
            xtrain = self.bootstrapped_xsamples[i,:]     #training samples for ith tree
            ytrain = self.bootstrapped_ysamples[i]
            classifier.fit(xtrain, ytrain)
            print(f"Completed training classifier {i}")
            xtest = X[notused_idx[i]]         #sampling X, y using the not used indexes to create test set
            ytest = y[notused_idx[i]]         #for each classifier
            yhat = classifier.predict(xtest)
            accuracy.append(np.sum(yhat==ytest)/ytest.shape[0])      #Calculating accuracy
        print("\nAccuracy of classifiers using oob training data: ", accuracy)
        print("Average Accuracy of classifiers: ", np.mean(accuracy))
        
            
    def predict(self, X_test):
        #Defining y_predicted matrix to hold predictions from each classifier
        y_predicted = np.zeros((self.B, X_test.shape[0]))
        #Getting predictions from each classifier
        for b, classifier in enumerate(self.classifiers):  
            y_predicted[b,:] = classifier.predict(X_test)
        #Getting the most common prediction (if tie, then smaller value)
        #print(y_predicted)
        y_predicted = stats.mode(y_predicted, axis=0)[0][0]  #along axis=0 -> meaning across classifiers
        return y_predicted
        
    def _bootstrap_samples(self, X, y, with_replacement=True):
        notused_sample_idx_alltrainigsets = []
        for b in range(0,self.B):      #For every Decision Tree Classifier
            used_sample_idx = []       #reset used index list for every tree
            for sample in range(0, self.sample_size):     #For every training sample for the tree
                sample_idx = np.random.randint(0, X.shape[0])    #Get a random sample index
                if with_replacement==False:
                    while (sample_idx in used_sample_idx):             #Loop until unused index found
                        sample_idx = np.random.randint(0, X.shape[0])
                used_sample_idx.append(sample_idx)                          #Keep a list of used indexes
                self.bootstrapped_xsamples[b, sample, :] = X[sample_idx]
                self.bootstrapped_ysamples[b, sample] = y[sample_idx]
            #for bootstrap_ratio = 1.0 without replacement, all samples are used for training each tree
            notused_sample_idx_alltrainigsets.append( 
                [idx for idx in range(0, self.sample_size) if idx not in used_sample_idx] ) 
        return notused_sample_idx_alltrainigsets        #For out of bag evaluation later
        

In [14]:
data = load_iris()
X,y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
model = bagging(b=5, bootstrap_ratio = 0.90)

In [15]:
model.fit(X_train, y_train, with_replacement=True)
yhat=model.predict(X_test)
print("\nyhat: ", yhat)
print("\nAccuracy with Bagging: ")
print(classification_report(y_test, yhat))

Bagging with replacement:  True
Using sample size:  94
not used indexes:  [[1, 2, 6, 7, 10, 15, 16, 20, 21, 25, 26, 27, 32, 33, 35, 36, 39, 41, 43, 45, 48, 49, 51, 54, 57, 58, 59, 60, 61, 63, 67, 68, 71, 72, 78, 81, 86, 87, 91], [1, 3, 5, 6, 10, 16, 19, 24, 25, 26, 28, 30, 33, 40, 42, 43, 45, 46, 52, 54, 56, 59, 61, 62, 65, 68, 69, 70, 72, 73, 74, 78, 79, 84, 85, 88, 89, 90], [0, 1, 4, 5, 7, 8, 9, 11, 15, 22, 25, 28, 29, 32, 35, 36, 41, 44, 46, 47, 49, 52, 53, 54, 56, 57, 61, 64, 67, 68, 69, 75, 77, 79, 81, 83, 85, 86, 88, 93], [1, 3, 5, 6, 8, 9, 11, 13, 19, 20, 21, 22, 29, 31, 32, 33, 36, 37, 38, 46, 47, 49, 50, 52, 54, 56, 57, 59, 63, 67, 69, 70, 76, 77, 79, 82, 83, 84, 87, 93], [0, 4, 10, 12, 15, 16, 17, 18, 21, 22, 24, 29, 30, 32, 33, 34, 35, 39, 40, 42, 46, 47, 58, 60, 63, 65, 66, 67, 68, 70, 72, 73, 75, 79, 82, 83, 91]]
Completed training classifier 0
Completed training classifier 1
Completed training classifier 2
Completed training classifier 3
Completed training classifier 4

A

In [16]:
model.fit(X_train, y_train, with_replacement=False)
yhat=model.predict(X_test)
print("\nyhat: ", yhat)
print("\nAccuracy with Pasting: ")
print(classification_report(y_test, yhat))

Bagging with replacement:  False
Using sample size:  94
not used indexes:  [[10, 27, 67, 69, 70, 73, 83, 87, 88, 90], [3, 15, 20, 24, 45, 59, 72, 92, 93], [0, 3, 24, 34, 39, 54, 77, 79], [7, 18, 20, 30, 49, 66, 78, 88, 90], [16, 24, 32, 40, 52, 71, 74]]
Completed training classifier 0
Completed training classifier 1
Completed training classifier 2
Completed training classifier 3
Completed training classifier 4

Accuracy of classifiers using oob training data:  [0.8, 1.0, 1.0, 1.0, 0.7142857142857143]
Average Accuracy of classifiers:  0.9028571428571428

yhat:  [2. 1. 0. 2. 1. 0. 0. 2. 0. 1. 0. 2. 0. 1. 0. 0. 1. 0. 0. 2. 1. 1. 0. 2.
 1. 0. 1. 2. 2. 1. 0. 1. 0. 2. 1. 0. 1. 0. 2. 1. 1. 2. 1. 1. 1.]

Accuracy with Pasting: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.94      0.94      0.94        18
           2       0.91      0.91      0.91        11

    accuracy                           0.96        4

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier()

classifier1 = BaggingClassifier(tree, n_estimators=5, max_samples=0.99)

classifier1.fit(X_train, y_train)
yhat = classifier1.predict(X_test)
print("Bagging Accuracy: ")
print(classification_report(y_test, yhat))

Bagging Accuracy: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.89      0.94        18
           2       0.85      1.00      0.92        11

    accuracy                           0.96        45
   macro avg       0.95      0.96      0.95        45
weighted avg       0.96      0.96      0.96        45



In [20]:
tree2 = DecisionTreeClassifier()

#pasting
parameters = {'base_estimator': tree2, 'n_estimators':5, 'max_samples':0.99, 'bootstrap':False}
classifier2 = BaggingClassifier(**parameters)

classifier2.fit(X_train, y_train)
yhat = classifier2.predict(X_test)
print("Pasting Accuracy: ")
print(classification_report(y_test, yhat))

Pasting Accuracy: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.94      0.97        18
           2       0.92      1.00      0.96        11

    accuracy                           0.98        45
   macro avg       0.97      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



In [21]:
for n_trees in [4, 5, 6, 7, 8, 9, 10]:
    for sample_ratio in [0.6, 0.7, 0.8, 0.9, 1.0]:
        model=bagging(b=n_trees, bootstrap_ratio=sample_ratio )
        model.fit(X_train, y_train, with_replacement=True)
        yhat=model.predict(X_test)
        print("\nyhat: ", yhat)
        print(f"\nAccuracy with Bagging (trees = {n_trees}, bootstrap_ratio={sample_ratio}): ")
        print(classification_report(y_test, yhat))


Bagging with replacement:  True
Using sample size:  63
not used indexes:  [[0, 1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 16, 19, 21, 22, 26, 27, 28, 36, 37, 38, 39, 40, 44, 46, 47, 50, 52, 53, 54, 57, 58, 59, 60, 61], [0, 1, 2, 3, 4, 5, 10, 11, 12, 13, 14, 16, 18, 22, 23, 24, 25, 26, 27, 28, 31, 34, 38, 39, 40, 44, 45, 47, 49, 50, 52, 53, 54, 55, 57, 59, 61, 62], [0, 1, 2, 4, 6, 7, 11, 12, 15, 17, 19, 20, 24, 26, 27, 29, 30, 31, 33, 36, 37, 38, 39, 40, 41, 42, 43, 45, 47, 49, 52, 54, 55, 57, 58, 62], [1, 6, 7, 8, 11, 15, 16, 17, 18, 20, 21, 23, 24, 25, 27, 28, 29, 30, 31, 32, 34, 37, 38, 40, 41, 42, 43, 44, 45, 46, 51, 53, 54, 57, 58, 59, 60, 61]]
Completed training classifier 0
Completed training classifier 1
Completed training classifier 2
Completed training classifier 3

Accuracy of classifiers using oob training data:  [0.918918918918919, 0.8157894736842105, 0.8611111111111112, 0.7368421052631579]
Average Accuracy of classifiers:  0.8331654022443497

yhat:  [2. 1. 0. 1. 1. 0. 0. 2. 

Completed training classifier 4

Accuracy of classifiers using oob training data:  [0.9714285714285714, 1.0, 0.9736842105263158, 0.7575757575757576, 0.9487179487179487]
Average Accuracy of classifiers:  0.9302812976497188

yhat:  [2. 1. 0. 2. 2. 0. 0. 2. 0. 1. 0. 2. 0. 1. 0. 0. 1. 0. 0. 2. 1. 1. 0. 2.
 1. 0. 2. 2. 2. 1. 0. 1. 0. 2. 1. 0. 1. 0. 2. 1. 1. 2. 1. 2. 1.]

Accuracy with Bagging (trees = 5, bootstrap_ratio=0.9): 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.83      0.91        18
           2       0.79      1.00      0.88        11

    accuracy                           0.93        45
   macro avg       0.93      0.94      0.93        45
weighted avg       0.95      0.93      0.93        45

Bagging with replacement:  True
Using sample size:  105
not used indexes:  [[1, 4, 10, 12, 13, 17, 21, 22, 24, 29, 32, 33, 34, 36, 42, 43, 44, 45, 48, 50, 51, 53, 55, 57, 58, 61, 62, 63, 65, 66

not used indexes:  [[3, 4, 5, 6, 11, 12, 13, 14, 17, 18, 19, 20, 21, 22, 23, 27, 28, 29, 31, 33, 34, 36, 37, 43, 47, 53, 54, 56, 57, 61, 62, 67, 69, 70, 73, 74, 75, 79], [0, 4, 6, 8, 9, 10, 11, 13, 19, 20, 23, 24, 25, 28, 33, 34, 39, 41, 44, 45, 49, 50, 51, 53, 55, 56, 60, 61, 63, 64, 66, 67, 68, 69, 70, 71, 73, 76, 79, 82], [1, 2, 3, 7, 8, 9, 10, 11, 17, 18, 19, 26, 28, 29, 31, 35, 36, 39, 40, 41, 47, 49, 52, 53, 54, 58, 59, 61, 62, 63, 65, 72, 73, 74, 77, 79, 81], [1, 2, 3, 5, 9, 10, 11, 12, 15, 16, 18, 19, 20, 21, 22, 25, 27, 29, 39, 40, 41, 42, 44, 48, 50, 52, 53, 54, 55, 58, 60, 61, 66, 70, 74, 75, 77, 79, 83], [0, 1, 4, 6, 9, 10, 17, 18, 21, 22, 23, 25, 27, 28, 29, 30, 31, 32, 34, 39, 40, 41, 42, 43, 45, 46, 50, 51, 54, 55, 58, 59, 60, 61, 62, 65, 68, 69, 71, 74, 77, 82], [4, 5, 7, 10, 11, 12, 13, 15, 16, 17, 21, 23, 28, 32, 34, 39, 41, 49, 50, 54, 61, 63, 66, 68, 70, 71, 72, 76, 79, 82, 83], [0, 4, 5, 6, 8, 12, 13, 16, 17, 19, 20, 21, 22, 25, 35, 36, 37, 39, 44, 45, 46, 49, 50, 

Completed training classifier 1
Completed training classifier 2
Completed training classifier 3
Completed training classifier 4
Completed training classifier 5
Completed training classifier 6
Completed training classifier 7
Completed training classifier 8

Accuracy of classifiers using oob training data:  [0.9444444444444444, 0.9696969696969697, 0.9411764705882353, 0.9, 0.8108108108108109, 0.9166666666666666, 0.9473684210526315, 0.78125, 0.9428571428571428]
Average Accuracy of classifiers:  0.9060301029018779

yhat:  [2. 2. 0. 2. 2. 0. 0. 2. 0. 1. 0. 2. 0. 1. 0. 0. 1. 0. 0. 2. 1. 1. 0. 2.
 1. 0. 1. 2. 2. 1. 0. 1. 0. 1. 1. 0. 1. 0. 2. 1. 1. 2. 1. 1. 1.]

Accuracy with Bagging (trees = 9, bootstrap_ratio=0.6): 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       0.94      0.89      0.91        18
           2       0.83      0.91      0.87        11

    accuracy                           0.93        45
   macro 


Bagging with replacement:  True
Using sample size:  84
not used indexes:  [[4, 6, 7, 10, 13, 14, 15, 16, 17, 19, 20, 24, 27, 28, 30, 32, 33, 35, 37, 41, 42, 43, 54, 58, 59, 60, 61, 62, 65, 66, 69, 70, 73, 74, 77, 78, 79, 81], [0, 2, 3, 4, 6, 7, 11, 12, 15, 16, 22, 30, 33, 37, 38, 39, 42, 44, 47, 49, 52, 56, 58, 59, 63, 64, 65, 66, 70, 71, 73, 74, 77, 79, 80, 83], [1, 2, 7, 8, 9, 10, 14, 15, 17, 18, 19, 22, 23, 26, 28, 31, 40, 41, 51, 52, 56, 61, 62, 63, 64, 65, 67, 76, 77, 78], [4, 5, 7, 8, 12, 15, 17, 18, 19, 20, 22, 24, 31, 32, 33, 36, 38, 39, 41, 45, 50, 53, 54, 58, 60, 63, 64, 65, 66, 67, 68, 70, 71, 74, 78, 82, 83], [0, 1, 5, 16, 23, 26, 31, 32, 34, 35, 39, 40, 41, 42, 46, 47, 48, 49, 50, 53, 54, 55, 57, 58, 60, 61, 62, 63, 72, 74, 75, 77, 79, 80, 81], [2, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 26, 29, 34, 42, 43, 45, 47, 48, 49, 50, 51, 52, 53, 55, 56, 61, 62, 65, 72, 75, 77, 78, 81, 82], [5, 10, 12, 13, 15, 16, 20, 21, 22, 23, 25, 27, 28, 29, 30, 33, 34, 35, 36, 3